CS4001/4042 Assignment 1, Part B, Q4
---

Model degradation is a common issue faced when deploying machine learning models (including neural networks) in the real world. New data points could exhibit a different pattern from older data points due to factors such as changes in government policy or market sentiments. For instance, housing prices in Singapore have been increasing and the Singapore government has introduced 3 rounds of cooling measures over the past years (16 December 2021, 30 September 2022, 27 April 2023).

In such situations, the distribution of the new data points could differ from the original data distribution which the models were trained on. Recall that machine learning models often work with the assumption that the test distribution should be similar to train distribution. When this assumption is violated, model performance will be adversely impacted.  In the last part of this assignment, we will investigate to what extent model degradation has occurred.




---



---



Your co-investigators used a linear regression model to rapidly test out several combinations of train/test splits and shared with you their findings in a brief report attached in Appendix A below. You wish to investigate whether your deep learning model corroborates with their findings.

In [ ]:
!pip install alibi-detect

In [ ]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from alibi_detect.cd import TabularDrift

import torch
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)
from sklearn.metrics import r2_score


# from alibi_detect.cd import ChiSquareDrift, TabularDrift
# from alibi_detect.saving import save_detector, load_detector

> Evaluate your model from B1 on data from year 2022 and report the test R2.

In [ ]:
df = pd.read_csv('hdb_price_prediction.csv')

# TODO: Enter your code here

# Split dataset
train_df = df[df['year'] <= 2019]
test_df = df[df['year'] == 2022]

# Configure model
def configure_model():
	data_config = DataConfig(
		target=["resale_price"],
		continuous_cols=["dist_to_nearest_stn", "dist_to_dhoby", "degree_centrality", "eigenvector_centrality", "remaining_lease_years", "floor_area_sqm"],
		categorical_cols=["month", "town", "flat_model_type", "storey_range"],
	)

	trainer_config = TrainerConfig(batch_size=1024, max_epochs=50)

	model_config = CategoryEmbeddingModelConfig(
		task="regression",
		layers="10-50-1",  # Number of nodes in each layer: 10 attributes (input), 50 (hidden), 1 (output)
	)

	optimizer_config = OptimizerConfig(
			optimizer="Adam"
	)

	tabular_model = TabularModel(
		data_config=data_config,
		model_config=model_config,
		optimizer_config=optimizer_config,
		trainer_config=trainer_config,
	)

	return tabular_model

def train(model, train_df):
	model.fit(train=train_df)
	return model
	
def eval(model, test_df):
	result = model.evaluate(test_df)
	pred_df = model.predict(test_df)

	r2 = r2_score(pred_df["resale_price"].values, pred_df["resale_price_prediction"].values)
	return r2

mod = configure_model()
mod = train(mod, train_df)
r2_2022 = eval(mod, test_df)
print(r2_2022)

> Evaluate your model from B1 on data from year 2023 and report the test R2.

In [ ]:
# TODO: Enter your code here
test_df_2023 = df[df['year'] == 2023]

r2_2023 = eval(mod, test_df_2023)
print(r2_2023)

Model from B1, evaluated on data from year 2023:
R2 score = 0.09379716823865913


> Did model degradation occur for the deep learning model?


\# TODO: \<Enter your answer here\>

Yes, model degradation occurred for the deep learing model.
When tested on data from 2022, the model achieved a higher R2 score of 0.34506112201833483. But when tested on data from 2023, the R2 score dropped approximately three times to 0.09379716823865913



---



---



Model degradation could be caused by [various data distribution shifts](https://huyenchip.com/2022/02/07/data-distribution-shifts-and-monitoring.html#data-shift-types): covariate shift (features), label shift and/or concept drift (altered relationship between features and labels).
There are various conflicting terminologies in the [literature](https://www.sciencedirect.com/science/article/pii/S0950705122002854#tbl1). Let’s stick to this reference for this assignment.

> Using the **Alibi Detect** library, apply the **TabularDrift** function with the training data (year 2019 and before) used as the reference and **detect which features have drifted** in the 2023 test dataset. Before running the statistical tests, ensure you **sample 1000 data points** each from the train and test data. Do not use the whole train/test data. (Hint: use this example as a guide https://docs.seldon.io/projects/alibi-detect/en/stable/examples/cd_chi2ks_adult.html)


In [ ]:
# TODO: # TODO: Enter your code here

train_df = df[df['year'] <= 2019]

x_ref_df = train_df.drop(columns=['resale_price', 'nearest_stn', 'full_address', 'year'])
x_ref_1000 = x_ref_df[:1000]

test_df = df[df['year'] == 2023]
x_test_df = test_df.drop(columns=['resale_price', 'nearest_stn', 'full_address', 'year'])
x_test_1000 = x_test_df[:1000]

columns = x_ref_df.columns

# cat_map = {}
# for cat in ['month', 'town', 'flat_model_type', 'storey_range']:
# 	cat_map[cat] = train_df[cat].unique()
# cat_per_feature = {}
# cat_per_feature[0] = cat_map['month'].shape[0]
# cat_per_feature[1] = cat_map['town'].shape[0]
# cat_per_feature[6] = cat_map['flat_model_type'].shape[0]
# cat_per_feature[9] = cat_map['storey_range'].shape[0]

categories_per_feature = {f: None for f in range(len(columns))}

# print(cat_per_feature)
x_ref_1000_numpy = x_ref_1000.to_numpy()
x_test_1000_numpy = x_test_1000.to_numpy()


cd = TabularDrift(x_ref_1000_numpy, p_val=.05,categories_per_feature=categories_per_feature)

# print(x_ref_1000_numpy.shape, x_test_1000_numpy.shape)

preds = cd.predict(x_test_1000_numpy)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds['data']['is_drift']]))

In [ ]:
for f in range(cd.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    fname = columns[f]
    stat_val, p_val = preds['data']['distance'][f], preds['data']['p_val'][f]
    print(f'{fname} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

> Assuming that the flurry of housing measures have made an impact on the relationship between all the features and resale_price (i.e. P(Y|X) changes), which type of data distribution shift possibly led to model degradation?


\# TODO: \<Enter your answer here\>

> From your analysis via TabularDrift, which features contribute to this shift?


\# TODO: \<Enter your answer here\>

> Suggest 1 way to address model degradation and implement it, showing improved test R2 for year 2023.


\# TODO: \<Enter your answer here\>

In [ ]:
# TODO: Enter your code here

### Appendix A



Here are our results from a linear regression model. We used StandardScaler for continuous variables and OneHotEncoder for categorical variables.

While 2021 data can be predicted well, test R2 dropped rapidly for 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| Year <= 2020 | 2021     | 0.76    |
| Year <= 2020 | **2022**     | 0.41    |
| Year <= 2020 | **2023**     | **0.10**   |



Similarly, a model trained on 2017 data can predict 2018-2021 well (with slight degradation in performance for 2021), but drops drastically in 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2017         | 2018     | 0.90    |
|              | 2019     | 0.89    |
|              | 2020     | 0.87    |
|              | 2021     | 0.72    |
|              | **2022**     | **0.37**    |
|              | **2023**     | **0.09**    |

With the test set fixed at year 2021, training on data from 2017-2020 still works well on the test data, with minimal degradation. Training sets closer to year 2021 generally do better.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2020         | 2021     | 0.81    |
| 2019         | 2021     | 0.75    |
| 2018         | 2021     | 0.73    |
| 2017         | 2021     | 0.72    |